In [3]:
#added to use driver
import csv
import numpy as np
from neo4j import GraphDatabase
import pandas as pd
import operator
import re
import math
import random
from random import randint
import numpy as np
import argparse
import torch
import torch.autograd as autograd
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from LSTMTagger import LSTMTagger
from LSTMTrain import LSTMTrain
from LSTMEvaluation import LSTMEvaluation
from datetime import datetime

In [4]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "a"))

In [10]:
import networkx as nx
from node2vec import Node2Vec
def createembedding(file, query):
    print('Create the edge list')
    with driver.session() as session, open(file, "w") as edges_file:
        result = session.run(query)

        writer = csv.writer(edges_file, delimiter=" ")

        for row in result:
            writer.writerow([row["source"], row["target"]])
    
def createNode2Vec(file, EMBEDDING_FILENAME, EMBEDDING_MODEL_FILENAME):   
    # Create a graph
    graph=nx.read_edgelist(file, create_using = nx.DiGraph(), nodetype = None, data = [('weight', int)])

    # Precompute probabilities and generate walks
    node2vec = Node2Vec(graph, dimensions=10, walk_length=3, num_walks=20, workers=4)

    ## if d_graph is big enough to fit in the memory, pass temp_folder which has enough disk space
    # Note: It will trigger "sharedmem" in Parallel, which will be slow on smaller graphs
    #node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4, temp_folder="/mnt/tmp_data")
    # Embed
    model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)

    # Look for most similar nodes
    #model.wv.most_similar('2')  # Output node names are always strings

    # Save embeddings for later use
    model.wv.save_word2vec_format(EMBEDDING_FILENAME)

    # Save model for later use
    model.save(EMBEDDING_MODEL_FILENAME)

In [11]:
file = 'graph/movies.edgelist'
EMBEDDING_FILENAME = 'emb/node2vecmov.emb'
EMBEDDING_MODEL_FILENAME = 'emb/embdnode2vecmov.model'
movienode2vecquery = """\
        MATCH (m:Movie)--(other)
        RETURN id(m) AS source, id(other) AS target
        """

createembedding(file, movienode2vecquery)

Create the edge list


In [12]:
createNode2Vec(file, EMBEDDING_FILENAME, EMBEDDING_MODEL_FILENAME)

Computing transition probabilities: 100%|███████████████████████████████████████| 9980/9980 [00:00<00:00, 10788.50it/s]
C:\Users\meetn\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [97]:
file = 'graph/users.edgelist'
EMBEDDING_FILENAME = 'emb/node2vecuser.emb'
EMBEDDING_MODEL_FILENAME = 'emb/embdnode2vecuser.model'
usernode2vecquery = """\
        MATCH (u:User)--(other)
        RETURN id(u) AS source, id(other) AS target
        """
createembedding(file, usernode2vecquery)

Create the edge list


In [98]:
createNode2Vec(file, EMBEDDING_FILENAME, EMBEDDING_MODEL_FILENAME)

Computing transition probabilities: 100%|███████████████████████████████████████| 6257/6257 [00:00<00:00, 26249.72it/s]
C:\Users\meetn\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [5]:
import pandas as pd

movies_genres_query = """\
MATCH (genre:Genre)
WITH genre ORDER BY genre.name
WITH collect(id(genre)) AS genres
MATCH (m:Movie)<-[:IS_GENRE_OF]-(genre)
WITH genres, id(m) AS source, m.embedding AS embedding, collect(id(genre)) AS target
RETURN source, embedding, [g in genres | CASE WHEN g in target THEN 1 ELSE 0 END] AS genres
"""

with driver.session() as session:
    result = session.run(movies_genres_query)
    df = pd.DataFrame([dict(row) for row in result])

In [6]:
#What does our DataFrame look like at the moment?
df.head()

,source,embedding,genres
0,1,"[-0.0069524995, -0.0041386094, -0.005405706, 0...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1,2,"[0.004386477, 0.004046303, -0.006452442, -0.00...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
2,3,"[-0.0076084025, 0.0017854427, 0.0066224835, 0....","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,"[-0.0011135519, 0.003364261, 0.0034164581, -0....","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,"[0.0019104508, -0.005871291, -0.0030465412, -0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [7]:
train_index = int(len(df) * 0.9)
train_data = df[:train_index]
test_data = df[train_index:]

train_x = train_data.ix[:, "embedding"]
train_x = pd.DataFrame(np.array([np.array(item) for item in train_x.values]))
train_x.columns = [str(col) for col in train_x.columns.get_values()]

train_y = train_data.ix[:, 'genres']

# separate test data
test_x = test_data.ix[:, "embedding"]
test_x = pd.DataFrame(np.array([np.array(item) for item in test_x.values]))
test_x.columns = [str(col) for col in train_x.columns.get_values()]

test_y = test_data.ix[:, 'genres']

C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  import sys
C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.il

Building Tensorflow model

In [8]:
import tensorflow as tf
from tensorflow.python.feature_column import feature_column_lib
from tensorflow.python.training import training_util
from tensorflow.python.training.ftrl import FtrlOptimizer

create a simple multi label classifier because a movie can have more than one genre associated with it

In [9]:
def train_input_fn(features, labels, batch_size):
    labels = tf.constant(np.array([np.array(item)
                                   for item in labels.values]))
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    return dataset.shuffle(1000).repeat().batch(batch_size)


def eval_input_fn(features, labels, batch_size):
    labels = tf.constant(np.array([np.array(item)
                                   for item in labels.values]))

    features = dict(features)
    inputs = (features, labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    assert batch_size is not None, "batch_size must not be None"
    return dataset.batch(batch_size)

In [10]:
def model_fn(features, labels, mode, config):
    def train_op_fn(loss):
        opt = FtrlOptimizer(learning_rate=LEARNING_RATE)
        return opt.minimize(loss,
                            global_step=training_util.get_global_step())

    def logit_fn(features):
        return feature_column_lib.linear_model(
            features=features,
            feature_columns=feature_columns,
            units=head.logits_dimension)

    return head.create_estimator_spec(
        features=features,
        mode=mode,
        logits=logit_fn(features=features),
        labels=labels,
        train_op_fn=train_op_fn)

In [11]:
def predict_input_fn(df_data, num_epochs):
        return tf.estimator.inputs.pandas_input_fn(
            x=df_data,
            num_epochs=num_epochs,
           shuffle=False
        )

In [14]:

feature_columns = [tf.feature_column.numeric_column(key=key)
                   for key in train_x.keys()]

LEARNING_RATE = 0.3
loss_reduction = tf.losses.Reduction.SUM_OVER_BATCH_SIZE

head = tf.compat.v1.estimator.MultiLabelHead(18, weight_column=None, label_vocabulary=None,loss_reduction=loss_reduction)

classifier = tf.estimator.Estimator(model_fn=model_fn)

classifier.train(
    input_fn=lambda: train_input_fn(train_x, train_y, 100),
    steps=2000)

eval_result = classifier.evaluate(
    input_fn=lambda: eval_input_fn(test_x, test_y, 100))

print(eval_result)
print('\nTest set AUC: {auc:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\meetn\\AppData\\Local\\Temp\\tmp9mijl_18', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F536BFF608>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:C

In [15]:
genres_query = """\
MATCH (genre:Genre)
WITH genre ORDER BY genre.name
RETURN collect(genre.name) AS genres
"""

with driver.session() as session:
    result = session.run(genres_query)
    genres = result.peek()["genres"]


movies_genres_predict_query = """\
MATCH (genre:Genre)
WITH genre ORDER BY genre.name
WITH collect(id(genre)) AS genres
MATCH (m:Movie)<-[:IS_GENRE_OF]-(genre)
WITH genres, m.title AS movie, id(m) AS source, m.embedding AS embedding, collect(id(genre)) AS target
RETURN source, movie, embedding, [g in genres | CASE WHEN g in target THEN 1 ELSE 0 END] AS genres
ORDER BY rand()
LIMIT 3
"""

with driver.session() as session:
    result = session.run(movies_genres_predict_query)
    predict_df = pd.DataFrame([dict(row) for row in result])
    
print(predict_df.head())
expected_df = predict_df[["genres", "source", "movie"]]

predict_x = predict_df.ix[:, "embedding"]
predict_x = pd.DataFrame(np.array([np.array(item) for item in predict_x.values]))
predict_x.columns = [str(col) for col in predict_x.columns.get_values()]

predict_x

   source                                              movie  \
0    2787  Decline of Western Civilization Part II: The M...   
1    3785                                      Stolen Summer   
2    6993             Revenge of the Nerds IV: Nerds in Love   

                                           embedding  \
0  [8.0696545e-06, 0.0002127382, 0.00085373764, -...   
1  [0.0045347894, -0.0042127357, 0.0063719163, -0...   
2  [0.000993466, -0.006794362, 0.0065342835, -0.0...   

                                              genres  
0  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...  
2  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...  


C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.000008,0.000213,0.000854,-0.006944,0.000104,-0.005319,0.003169,0.002138,0.006348,-0.002914,...,-0.003319,-0.005768,-0.004754,-0.004165,0.007019,-0.005927,-0.001096,-0.003606,0.007288,-0.000287
1,0.004535,-0.004213,0.006372,-0.005838,-0.006285,-0.004112,0.002036,-0.000927,0.003899,0.005837,...,0.004062,-0.006845,-0.002564,-0.003860,0.000222,-0.007590,0.004161,-0.003131,0.006485,0.000765
2,0.000993,-0.006794,0.006534,-0.000412,-0.004108,0.004736,-0.004326,-0.000629,0.006575,-0.006136,...,-0.001432,0.004912,-0.002654,0.001216,-0.005549,-0.002012,-0.003434,0.007617,0.007020,0.006649


In [17]:

def predict_input_fn(predict_x, num):
    return  tf.data.Dataset.from_tensors(dict(predict_x))

from tensorflow_core.estimator import inputs

predictions = classifier.predict(input_fn=lambda: predict_input_fn(predict_x, 100))
#user movie 
for pred_dict, expec in zip(predictions, expected_df.as_matrix()):
    expected, source, movie = expec
    print("Movie: {0}".format(movie))
    for idx, label in enumerate(expected):
        print(label, genres[idx], pred_dict["probabilities"][idx])
    print("--")

C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\meetn\AppData\Local\Temp\tmp9mijl_18\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Movie: Decline of Western Civilization Part II: The Metal Years, The
0 Action 0.18960461
0 Adventure 0.12896603
0 Animation 0.054834902
0 Children's 0.011577904
0 Comedy 0.38235083
0 Crime 0.12811694
1 Documenta